## Portfolio Exercise: Starbucks
<br>

<img src="https://opj.ca/wp-content/uploads/2018/02/New-Starbucks-Logo-1200x969.jpg" width="200" height="200">
<br>
<br>
 
#### Background Information

The dataset you will be provided in this portfolio exercise was originally used as a take-home assignment provided by Starbucks for their job candidates. The data for this exercise consists of about 120,000 data points split in a 2:1 ratio among training and test files. In the experiment simulated by the data, an advertising promotion was tested to see if it would bring more customers to purchase a specific product priced at $10. Since it costs the company 0.15 to send out each promotion, it would be best to limit that promotion only to those that are most receptive to the promotion. Each data point includes one column indicating whether or not an individual was sent a promotion for the product, and one column indicating whether or not that individual eventually purchased that product. Each individual also has seven additional features associated with them, which are provided abstractly as V1-V7.

#### Optimization Strategy

Your task is to use the training data to understand what patterns in V1-V7 to indicate that a promotion should be provided to a user. Specifically, your goal is to maximize the following metrics:

* **Incremental Response Rate (IRR)** 

IRR depicts how many more customers purchased the product with the promotion, as compared to if they didn't receive the promotion. Mathematically, it's the ratio of the number of purchasers in the promotion group to the total number of customers in the purchasers group (_treatment_) minus the ratio of the number of purchasers in the non-promotional group to the total number of customers in the non-promotional group (_control_).

$$ IRR = \frac{purch_{treat}}{cust_{treat}} - \frac{purch_{ctrl}}{cust_{ctrl}} $$


* **Net Incremental Revenue (NIR)**

NIR depicts how much is made (or lost) by sending out the promotion. Mathematically, this is 10 times the total number of purchasers that received the promotion minus 0.15 times the number of promotions sent out, minus 10 times the number of purchasers who were not given the promotion.

$$ NIR = (10\cdot purch_{treat} - 0.15 \cdot cust_{treat}) - 10 \cdot purch_{ctrl}$$

For a full description of what Starbucks provides to candidates see the [instructions available here](https://drive.google.com/open?id=18klca9Sef1Rs6q8DW4l7o349r8B70qXM).

Below you can find the training data provided.  Explore the data and different optimization strategies.

#### How To Test Your Strategy?

When you feel like you have an optimization strategy, complete the `promotion_strategy` function to pass to the `test_results` function.  
From past data, we know there are four possible outomes:

Table of actual promotion vs. predicted promotion customers:  

<table>
<tr><th></th><th colspan = '2'>Actual</th></tr>
<tr><th>Predicted</th><th>Yes</th><th>No</th></tr>
<tr><th>Yes</th><td>I</td><td>II</td></tr>
<tr><th>No</th><td>III</td><td>IV</td></tr>
</table>

The metrics are only being compared for the individuals we predict should obtain the promotion – that is, quadrants I and II.  Since the first set of individuals that receive the promotion (in the training set) receive it randomly, we can expect that quadrants I and II will have approximately equivalent participants.  

Comparing quadrant I to II then gives an idea of how well your promotion strategy will work in the future. 

Get started by reading in the data below.  See how each variable or combination of variables along with a promotion influences the chance of purchasing.  When you feel like you have a strategy for who should receive a promotion, test your strategy against the test dataset used in the final `test_results` function.

In [255]:
# load in packages
from itertools import combinations

from test_results import test_results, score
import numpy as np
import pandas as pd
import scipy as sp
import sklearn as sk

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.linear_model import LinearRegression

import matplotlib.pyplot as plt
import seaborn as sb
%matplotlib inline

# load in the data
train_data = pd.read_csv('./training.csv')
train_data.head()

,ID,Promotion,purchase,V1,V2,V3,V4,V5,V6,V7
0,1,No,0,2,30.443518,-1.165083,1,1,3,2
1,3,No,0,3,32.159350,-0.645617,2,3,2,2
2,4,No,0,2,30.431659,0.133583,1,1,4,2
3,5,No,0,0,26.588914,-0.212728,2,1,4,2
4,8,Yes,0,3,28.044332,-0.385883,1,1,2,2


In [256]:
# Cells for you to work and document as necessary - 
# definitely feel free to add more cells as you need

## Assesment

First we will look at the data to check for cleaning issues.

In [257]:
#Split into treat and control
df_treat = train_data[train_data['Promotion']=='Yes']
df_control = train_data[train_data['Promotion']=='No']

In [258]:
# Show summary 
train_data.describe()

,ID,purchase,V1,V2,V3,V4,V5,V6,V7
count,84534.000000,84534.000000,84534.000000,84534.000000,84534.000000,84534.000000,84534.000000,84534.000000,84534.000000
mean,62970.972413,0.012303,1.500662,29.973600,0.000190,1.679608,2.327643,2.502898,1.701694
std,36418.440539,0.110234,0.868234,5.010626,1.000485,0.466630,0.841167,1.117349,0.457517
min,1.000000,0.000000,0.000000,7.104007,-1.684550,1.000000,1.000000,1.000000,1.000000
25%,31467.250000,0.000000,1.000000,26.591501,-0.905350,1.000000,2.000000,2.000000,1.000000
50%,62827.500000,0.000000,2.000000,29.979744,-0.039572,2.000000,2.000000,3.000000,2.000000
75%,94438.750000,0.000000,2.000000,33.344593,0.826206,2.000000,3.000000,4.000000,2.000000
max,126184.000000,1.000000,3.000000,50.375913,1.691984,2.000000,4.000000,4.000000,2.000000


In [259]:
df_treat.describe()

,ID,purchase,V1,V2,V3,V4,V5,V6,V7
count,42364.000000,42364.000000,42364.000000,42364.000000,42364.000000,42364.000000,42364.000000,42364.000000,42364.000000
mean,62847.675385,0.017019,1.505028,29.970338,0.007446,1.678949,2.324662,2.504225,1.701185
std,36299.420024,0.129344,0.868226,5.024443,0.998020,0.466886,0.840242,1.117279,0.457744
min,8.000000,0.000000,0.000000,7.104007,-1.684550,1.000000,1.000000,1.000000,1.000000
25%,31387.500000,0.000000,1.000000,26.571116,-0.818772,1.000000,2.000000,2.000000,1.000000
50%,62717.000000,0.000000,2.000000,29.989227,0.047006,2.000000,2.000000,3.000000,2.000000
75%,93960.500000,0.000000,2.000000,33.332278,0.826206,2.000000,3.000000,4.000000,2.000000
max,126181.000000,1.000000,3.000000,49.823708,1.691984,2.000000,4.000000,4.000000,2.000000


In [260]:
df_control.describe()

,ID,purchase,V1,V2,V3,V4,V5,V6,V7
count,42170.000000,42170.000000,42170.000000,42170.000000,42170.000000,42170.000000,42170.000000,42170.000000,42170.000000
mean,63094.836661,0.007565,1.496277,29.976876,-0.007099,1.680270,2.330638,2.501565,1.702205
std,36537.629680,0.086646,0.868229,4.996765,1.002915,0.466377,0.842093,1.117431,0.457294
min,1.000000,0.000000,0.000000,8.939305,-1.684550,1.000000,1.000000,1.000000,1.000000
25%,31528.250000,0.000000,1.000000,26.608910,-0.905350,1.000000,2.000000,2.000000,1.000000
50%,62922.000000,0.000000,1.000000,29.969732,-0.039572,2.000000,2.000000,3.000000,2.000000
75%,94893.750000,0.000000,2.000000,33.354970,0.826206,2.000000,3.000000,4.000000,2.000000
max,126184.000000,1.000000,3.000000,50.375913,1.691984,2.000000,4.000000,4.000000,2.000000


In [261]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 84534 entries, 0 to 84533
Data columns (total 10 columns):
ID           84534 non-null int64
Promotion    84534 non-null object
purchase     84534 non-null int64
V1           84534 non-null int64
V2           84534 non-null float64
V3           84534 non-null float64
V4           84534 non-null int64
V5           84534 non-null int64
V6           84534 non-null int64
V7           84534 non-null int64
dtypes: float64(2), int64(7), object(1)
memory usage: 6.4+ MB


In [262]:
train_data.isnull().sum()

ID           0
Promotion    0
purchase     0
V1           0
V2           0
V3           0
V4           0
V5           0
V6           0
V7           0
dtype: int64

-  We will change V1, V4, V5, V6, and V7 to objects

In [263]:
df_treat.head()

,ID,Promotion,purchase,V1,V2,V3,V4,V5,V6,V7
4,8,Yes,0,3,28.044332,-0.385883,1,1,2,2
8,15,Yes,0,2,31.930423,0.393317,2,3,1,2
10,17,Yes,0,1,32.770916,-1.511395,2,1,4,1
12,20,Yes,0,1,36.957009,0.133583,2,3,1,1
14,22,Yes,0,3,36.911714,-0.905350,2,2,4,1


### IRR Incremental Response Rate & NIR Net Incremental Revenue

In [264]:
# Create function to calculate IRR and NIR
def cal_irr ():
    treat = train_data[train_data['Promotion']=='Yes']
    control = train_data[train_data['Promotion']=='No']
    
    purch_t = treat['purchase'].sum()
    purch_c = control['purchase'].sum()
    cust_t = treat.shape[0]
    cust_c = control.shape[0]
    
    irr = (purch_t/cust_t) - (purch_c/cust_c)
    nir = (10* purch_t - 0.15 * cust_t) - 10* purch_c
    
    return irr, nir

irr, nir = cal_irr()
print('Overall IRR: ',irr)
print('Overall NIR: ',nir)

Overall IRR:  0.00945454781977
Overall NIR:  -2334.6


### Compare Correlation

First we will see if there are any differences between the groups as they relate to purchases.  We will compare correlation to purchase between treatment and control groups.

In [265]:
corr = df_treat.corr()
corr.style.background_gradient(cmap='coolwarm')

,ID,purchase,V1,V2,V3,V4,V5,V6,V7
ID,1,0.00554927,-0.00225473,-0.00267991,0.00130088,-0.0124178,-0.000783369,-0.00381238,-0.00170614
purchase,0.00554927,1,-0.0046507,-0.0028849,-0.0170124,0.0533479,0.0117111,-0.00204939,-0.0050054
V1,-0.00225473,-0.0046507,1,-0.00197263,0.00941734,0.00185669,-0.00502038,-0.00773587,0.0025034
V2,-0.00267991,-0.0028849,-0.00197263,1,-0.0024402,-0.0017628,-0.00365992,-0.00254373,-0.00479457
V3,0.00130088,-0.0170124,0.00941734,-0.0024402,1,-0.000656069,-0.00197287,0.00790547,-0.000367563
V4,-0.0124178,0.0533479,0.00185669,-0.0017628,-0.000656069,1,0.000585717,-0.00249032,-0.00311294
V5,-0.000783369,0.0117111,-0.00502038,-0.00365992,-0.00197287,0.000585717,1,0.00185783,0.00226483
V6,-0.00381238,-0.00204939,-0.00773587,-0.00254373,0.00790547,-0.00249032,0.00185783,1,-0.00401614
V7,-0.00170614,-0.0050054,0.0025034,-0.00479457,-0.000367563,-0.00311294,0.00226483,-0.00401614,1


In [266]:
corr = df_control.corr()
corr.style.background_gradient(cmap='coolwarm')

,ID,purchase,V1,V2,V3,V4,V5,V6,V7
ID,1,0.00964839,-0.00610452,-0.00283807,0.00532656,-0.00879372,0.00102069,0.000947574,0.00506722
purchase,0.00964839,1,-0.00608776,8.99842e-05,0.0083248,0.0023437,0.0027712,-0.000979527,0.00478587
V1,-0.00610452,-0.00608776,1,-0.00114483,-0.00149867,0.00329733,0.00267296,0.000397092,0.000373074
V2,-0.00283807,8.99842e-05,-0.00114483,1,0.00479797,0.00750839,0.00158474,-0.00578359,0.000235235
V3,0.00532656,0.0083248,-0.00149867,0.00479797,1,0.00818707,-0.00144816,0.00100714,0.00265563
V4,-0.00879372,0.0023437,0.00329733,0.00750839,0.00818707,1,0.00573575,0.00762655,-0.00324293
V5,0.00102069,0.0027712,0.00267296,0.00158474,-0.00144816,0.00573575,1,-0.00794658,0.00487422
V6,0.000947574,-0.000979527,0.000397092,-0.00578359,0.00100714,0.00762655,-0.00794658,1,0.00123697
V7,0.00506722,0.00478587,0.000373074,0.000235235,0.00265563,-0.00324293,0.00487422,0.00123697,1


##### Control group
-  There did not appreat to be any variables that had a positive or negative correlation to purchases.

##### Treatment group
- V4 had the strongest positive correlation to purchases.
- V5 had a small positive correlation to purchases.
- V3 had a small negative correlation to purchases.

##### Action decision
We will change ['V1','V4','V5','V6','V7'] to objects because we cannot tell if they are ment to be levels or numbers.

In [267]:
#Change int cols to objects
for col in ['V1','V4','V5','V6','V7']:
    df_treat[col] = df_treat[col].apply(str)
    df_control[col] = df_control[col].apply(str)   

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


In [268]:
print('Total Purchases in treatment group: ', df_treat['purchase'].sum())
print('Total Purchases in control group: ', df_control['purchase'].sum())
print('Total number of customers: ', train_data.shape[0])

Total Purchases in treatment group:  721
Total Purchases in control group:  319
Total number of customers:  84534


### Modeling

Multiple models were tested(linear regression logistic regression, random forest, adaboost, ect) and would return a '0' for 100% of all predictions.  For this reason we chose to use a linear regression and set a lower threshold to predict a purchase.  We want our model to have a higher recall and still maintain precision, therefore we will maximize our F2 score to select our threshold to predict a purchase.


In [269]:
#Workspace for setting up model
#Set X and y
X = df_treat[['V1','V2','V3','V4','V5','V6','V7']]

y = df_treat['purchase']

# Split into test and train
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

# Fit and predict model
model = LinearRegression()
model.fit(X_train, y_train)
y_preds = model.predict(X_test)

# optimize f1_score
for i in [0.025,0.023,0.022,.021,.02,.019]:
    y_test_predictions_high_recall = [1 if (x >= i) else 0 for x in y_preds]
    print('Threshold for promotion: ',i)
    print('Accuracy: ', accuracy_score(y_test,y_test_predictions_high_recall))
    print('Precision: ', precision_score(y_test,y_test_predictions_high_recall))
    print('Recall: ', recall_score(y_test,y_test_predictions_high_recall))
    print('F1 Score: ', f1_score(y_test,y_test_predictions_high_recall))
    print(' ')
    

Threshold for promotion:  0.025
Accuracy:  0.947786281382
Precision:  0.0276134122288
Recall:  0.0557768924303
F1 Score:  0.0369393139842
 
Threshold for promotion:  0.023
Accuracy:  0.849724626279
Precision:  0.0266120777892
Recall:  0.207171314741
F1 Score:  0.0471655328798
 
Threshold for promotion:  0.022
Accuracy:  0.774837279165
Precision:  0.0277143788719
Recall:  0.338645418327
F1 Score:  0.0512356841471
 
Threshold for promotion:  0.021
Accuracy:  0.690007867821
Precision:  0.0275399213145
Recall:  0.474103585657
F1 Score:  0.0520559930009
 
Threshold for promotion:  0.02
Accuracy:  0.594306558901
Precision:  0.0265502183406
Recall:  0.605577689243
F1 Score:  0.0508701472557
 
Threshold for promotion:  0.019
Accuracy:  0.509119519348
Precision:  0.025817555938
Recall:  0.717131474104
F1 Score:  0.0498407863769
 


- Here a threshold of '0.021' had the best F1 score so it will be used in our promotion strategy

In [270]:
def promotion_strategy(df):
    '''
    INPUT 
    df - a dataframe with *only* the columns V1 - V7 (same as train_data)

    OUTPUT
    promotion_df - np.array with the values
                   'Yes' or 'No' related to whether or not an 
                   individual should recieve a promotion 
                   should be the length of df.shape[0]
                
    Ex:
    INPUT: df
    
    V1	V2	  V3	V4	V5	V6	V7
    2	30	-1.1	1	1	3	2
    3	32	-0.6	2	3	2	2
    2	30	0.13	1	1	4	2
    
    OUTPUT: promotion
    
    array(['Yes', 'Yes', 'No'])
    indicating the first two users would recieve the promotion and 
    the last should not.
    '''
    #X = df_treat[['V1','V2','V3','V4','V5','V6','V7']]
    X = df_treat[['V3','V4','V5']]
    y = df_treat['purchase']

    # Fit and predict model
    model = LinearRegression()
    model.fit(X, y)
    
    for col in ['V1','V4','V5','V6','V7']:
        df[col] = df[col].apply(str)
    
    #y_preds = model.predict(df[['V1','V2','V3','V4','V5','V6','V7']])
    y_preds = model.predict(df[['V3','V4','V5']])
    proms = [1 if (x >= 0.021) else 0 for x in y_preds]

    promotion = np.array(['No' if (x == 0) else 'Yes' for x in proms])
    
    print(Counter(promotion))
    return promotion

In [271]:
# This will test your results, and provide you back some information 
# on how well your promotion_strategy will work in practice

test_results(promotion_strategy)

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


Counter({'No': 24656, 'Yes': 16994})
Nice job!  See how well your strategy worked on our test data below!

Your irr with this strategy is 0.0175.

Your nir with this strategy is 216.05.
We came up with a model with an irr of 0.0188 and an nir of 189.45 on the test set.

 How did you do?


(0.017549843112077752, 216.04999999999995)